In [1]:
import pandas as pd
import ast
from ast import literal_eval
import shutil
import gzip
import os

In [2]:
Juegos = pd.read_csv("../02 - Desanidado de datos y limpieza/Juegos_ETL.csv")
Reviews = pd.read_csv("../02 - Desanidado de datos y limpieza/Reviews_ETL.csv")
Items = pd.read_csv("../02 - Desanidado de datos y limpieza/Items_ETL.csv")

C:\Users\atenc\AppData\Local\Temp\ipykernel_16032\1363233923.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  Items = pd.read_csv("../02 - Desanidado de datos y limpieza/Items_ETL.csv")


DATAFRAME PARA PRIMER FUNCIÓN

In [3]:
Juegos_hs = Items[["item_name", "playtime_forever"]]
Juegos_hs = Juegos_hs.groupby(['item_name'], as_index=False)['playtime_forever'].sum()
Juegos_hs

,item_name,playtime_forever
0,神明的一天世界(God's One Day World),6969
1,! That Bastard Is Trying To Steal Our Gold !,742
2,"""Glow Ball"" - The billiard puzzle game",21
3,#SelfieTennis,261
4,#SkiJump,0
...,...,...
10942,丛林守望者（Ranger of the jungle）,2255
10943,侠客风云传(Tale of Wuxia),19288
10944,大海战 Navy Field IV,126
10945,神楽道中記(KaguraDouchuuki),4337


In [4]:
# Unir los DataFrames en función de la columna común
DF_F1 = pd.merge(Juegos, Juegos_hs, left_on='Nombre_juego', right_on='item_name', how='inner')
DF_F1 = DF_F1[["Genero", "Año_Lanzamiento", "playtime_forever"]]
DF_F1["Genero"] = DF_F1["Genero"].apply(ast.literal_eval)
DF_F1 = DF_F1.explode('Genero')

# Agrupar el DataFrame por las columnas 'Genero' y 'Año' y sumar 'playtime_forever'
DF_F1 = DF_F1.groupby(['Genero', 'Año_Lanzamiento'], as_index=False)['playtime_forever'].sum()
DF_F1

,Genero,Año_Lanzamiento,playtime_forever
0,Action,1983,3582
1,Action,1984,384
2,Action,1988,16243
3,Action,1989,607
4,Action,1990,5
...,...,...,...
314,Web Publishing,2012,1947406
315,Web Publishing,2013,140060
316,Web Publishing,2015,332496
317,Web Publishing,2016,1


In [25]:
DF_F1.to_csv("../03 - Dataframe para funciones/PlayTimeGenre.csv", index = False)

DATAFRAME PARA SEGUNDA FUNCIÓN

In [5]:
# Unir los DataFrames en función de la columna común
DF_F2 = pd.merge(Juegos, Items, left_on='Nombre_juego', right_on='item_name', how='inner')
DF_F2 = DF_F2[["Genero", "Año_Lanzamiento", "user_id", "playtime_forever"]]
DF_F2["Genero"] = DF_F2["Genero"].apply(ast.literal_eval)
DF_F2 = DF_F2.explode('Genero')

# Agrupar el DataFrame por las columnas 'Genero' y 'Año' y sumar 'playtime_forever'
DF_F2 = DF_F2.groupby(['Genero', 'Año_Lanzamiento', "user_id"], as_index=False)['playtime_forever'].sum()
DF_F2.rename(columns={'user_id': 'Id_user', 'playtime_forever': 'Cant_hs_juego'}, inplace=True)
DF_F2

,Genero,Año_Lanzamiento,Id_user,Cant_hs_juego
0,Action,1983,76561197997129113,40
1,Action,1983,76561198006449409,78
2,Action,1983,76561198007478325,47
3,Action,1983,76561198009435936,0
4,Action,1983,76561198025071276,41
...,...,...,...,...
2948441,Web Publishing,2017,Eosoforcus,58
2948442,Web Publishing,2017,N47H4NI3L,1635
2948443,Web Publishing,2017,dirklah,796
2948444,Web Publishing,2017,kushziller,251


Correción del dataframe para segunda función

Como el dataframe para dicha función queda de casi 3 millones de lineas, voy a tomar una muestra aleatoria de 10% de dicha información a fines que funcione la API y se demuentre en la misma el correcto funcionamiento de la función planteada, debido a la limitación de RENDER en cuanto a la memoria RAM

In [4]:
df_corr = pd.read_csv("../03 - Dataframe para funciones/UserForGenre.csv")
df_muestra = df_corr.sample(frac=0.10, random_state=22)
df_muestra

,Genero,Año_Lanzamiento,Id_user,Cant_hs_juego
2924518,Strategy,2016,tiwizo,810
999921,Casual,2014,76561198050347115,898
1525991,Indie,2007,76561198084694331,12291
1777443,Indie,2015,omaiornoob,791
2231467,RPG,2014,DuckPencil,446
...,...,...,...,...
2521197,Simulation,2013,mincast2,823
42362,Action,2000,dr_evilcat,25
1338813,Free to Play,2014,DieMadchenschanderin,6590
2642653,Sports,2011,darkedge119,1820


In [5]:
df_muestra.to_csv("../03 - Dataframe para funciones/UserForGenre.csv", index = False)

DATAFRAME PARA TERCER FUNCIÓN

In [6]:
Items_detalle = Items.drop(columns={"user_id", "playtime_forever"}, axis=1)
Items_detalle = Items_detalle.drop_duplicates(subset='item_id')

In [7]:
DF_p = pd.merge(Reviews, Items_detalle, on='item_id')
DF_p = DF_p[["recommend", "posted", "sentiment_analysis", "item_name"]]
DF_p.head(2)

,recommend,posted,sentiment_analysis,item_name
0,True,2011,2,Killing Floor
1,True,2015,2,Killing Floor


In [51]:
DF_p.shape

(43402, 4)

In [8]:
# Filtrar y mantener solo las filas
DF_F3 = DF_p[(DF_p['recommend'] == True) & (DF_p['sentiment_analysis'] > 0)]
DF_F3 = DF_F3.drop(columns={"recommend", "sentiment_analysis"}, axis=1)
DF_F3 = DF_F3.groupby(['posted', 'item_name']).size().reset_index(name='Cant_recomendaciones')
DF_F3.rename(columns={'posted': 'Año', 'item_name': 'Nombre_juego', 'Cant_recomendaciones': 'Cant_rec' }, inplace=True)
DF_F3

,Año,Nombre_juego,Cant_rec
0,2010,Amnesia: The Dark Descent,1
1,2010,ArcaniA,1
2,2010,Audiosurf,1
3,2010,Chime,1
4,2010,Counter-Strike: Source,2
...,...,...,...
3948,2015,resident evil 4 / biohazard 4,1
3949,2015,sZone-Online,2
3950,2015,the static speaks my name,9
3951,2015,theHunter,13


In [44]:
DF_F3.to_csv("../03 - Dataframe para funciones/UsersRecommend.csv", index = False)

DATAFRAME PARA CUARTA FUNCIÓN

In [9]:
# Filtrar y mantener solo las filas
DF_F4 = DF_p[(DF_p['recommend'] == False) & (DF_p['sentiment_analysis'] == 0)]
DF_F4 = DF_F4.drop(columns={"recommend", "sentiment_analysis"}, axis=1)
DF_F4 = DF_F4.groupby(['posted', 'item_name']).size().reset_index(name='Cant_rec')
DF_F4.rename(columns={'posted': 'Año', 'item_name': 'Nombre_juego'}, inplace=True)
DF_F4

,Año,Nombre_juego,Cant_rec
0,2011,And Yet It Moves,2
1,2011,Dead Island,1
2,2011,Fable III,1
3,2011,Men of War: Vietnam,1
4,2012,Call of Duty: Black Ops II,1
...,...,...,...
936,2015,Zombie Army Trilogy,2
937,2015,Zombie Zoeds,1
938,2015,bit Dungeon II,1
939,2015,sZone-Online,1


In [12]:
DF_F4.to_csv("../03 - Dataframe para funciones/UsersNotRecommend.csv", index = False)

DATAFRAME PARA QUINTA FUNCIÓN

In [10]:
DF_F5 = pd.merge(DF_p, Juegos, left_on='item_name', right_on='Nombre_juego', how='inner')
DF_F5 = DF_F5[["Año_Lanzamiento", "sentiment_analysis"]]
DF_F5 = DF_F5.groupby(['Año_Lanzamiento', 'sentiment_analysis']).size().reset_index(name='Cant_reg')
DF_F5

,Año_Lanzamiento,sentiment_analysis,Cant_reg
0,1989,2,1
1,1991,2,1
2,1992,2,2
3,1993,0,1
4,1993,2,2
...,...,...,...
69,2016,1,134
70,2016,2,558
71,2017,0,259
72,2017,1,335


In [13]:
DF_F5.to_csv("../03 - Dataframe para funciones/sentiment_analysis.csv", index = False)

In [3]:
## comprimo los 3 archivos del paso anterior, por su tamaño
with open("../02 - Desanidado de datos y limpieza/Juegos_ETL.csv", 'rb') as f_in, gzip.open("../02 - Desanidado de datos y limpieza/Juegos_ETL.csv.gz", 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

os.remove("../02 - Desanidado de datos y limpieza/Juegos_ETL.csv")

In [4]:
with open("../02 - Desanidado de datos y limpieza/Reviews_ETL.csv", 'rb') as f_in, gzip.open("../02 - Desanidado de datos y limpieza/Reviews_ETL.csv.gz", 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

os.remove("../02 - Desanidado de datos y limpieza/Reviews_ETL.csv")

In [5]:
with open("../02 - Desanidado de datos y limpieza/Items_ETL.csv", 'rb') as f_in, gzip.open("../02 - Desanidado de datos y limpieza/Items_ETL.csv.gz", 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

os.remove("../02 - Desanidado de datos y limpieza/Items_ETL.csv")